# IXPE Spectro-Polarimetry in XSPEC
### Hands-on tutorial

**Focus:**
- What is special about IXPE Stokes data?
- How polarization is handled in XSPEC
- Simple analytical vs. physical table and analytical polarization models
- Practical workflow on loading the data and using polarization models in XSPEC


## What is different compared to standard spectroscopy?

In normal X-ray spectroscopy we fit a flux spectrum:

$$I(E)$$

With IXPE we effectively fit Stokes spectra:

$$I(E),\;Q(E),\;U(E)$$

Polarization degree (PD) and polarization angle (PA) are derived from:

$$\mathrm{PD}=\frac{\sqrt{Q^2+U^2}}{I}$$
$$\mathrm{PA}=\tfrac{1}{2}\arctan\left(\frac{U}{Q}\right)$$

XSPEC can forward-fold models that predict Stokes spectra through the instrument response.


## IXPE spectral–polarimetric products

Pipeline (IXPEOBSSIM or HEASOFT) produces:
- Stokes spectra I, Q, U
- Stokes background spectra
- Corresponding RMF, ARF and MRF response files

Key points:
- XSPEC can treat these as polarization-aware datasets
- The response propagates polarization sensitivity
- In practice you fit I/Q/U simultaneously

### Reduction / responses
We do not perform data reduction or binning live but let's point out basic steps with links to relevant information.

Typical steps:
1. Obtain IXPE data products (L1/L2)
   * IXPE [performed observations](https://ixpe.msfc.nasa.gov/for_scientists/asrun.html)
   * IXPE [long-term plan](https://ixpe.msfc.nasa.gov/for_scientists/ltp.html)
   * IXPE [weekly plan](https://ixpe.msfc.nasa.gov/for_scientists/weekly.html)
   * IXPE [data archive](https://heasarc.gsfc.nasa.gov/docs/ixpe/archive/) at HEASARC
   * IXPE [GO competition](https://heasarc.gsfc.nasa.gov/docs/ixpe/proposals/ixpe_prop.html) - usually late August or early September
2. Extract Stokes spectra
   * perform instrumental [background rejection](https://github.com/aledimarco/IXPE-background)
   * define source and background regions, e.g. with `ds9`
   * use `xselect` to filter event files
     - check light curve for variability 
     - exclude times with solar flares
     - additional filtering, e.g. choose periods according to the flux levels
     - choose weighting (NONE, SIMPLE, NEFF) and axtract spectra
3. Generate response files with `ixpecalcarf` → generates ARF and MRF for particular IXPE observation
   * [IXPE calibration](https://heasarc.gsfc.nasa.gov/docs/ixpe/caldb/)
4. [Contributed IXPE Software](https://heasarc.gsfc.nasa.gov/docs/ixpe/analysis/contributed.html)
   * [IXPEOBSSIM](https://ixpeobssim.readthedocs.io/en/latest/)
5. Bin the spectra with `grppha`
   * even for Stokes I (flux) it is advised to bin (e.g. to 30 bins in 2-8 keV) since the resolution of IXPE is ~0.5 keV
   * to be able to use `plot polfrac` or `plot polangle` in XSPEC, the binning has to be the same for all Stokes parameters
   * to show reasonable values of polarization degree and angle it may be useful to bin to fewer bins (3-10 bins in 2-8 keV)



In [1]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="astropy")
from pathlib import Path
from astropy.io import fits

# Example 1 data files
I_FILE = Path("example1/ixpe01250401_det1_evt2_v03_I-11bins.pha")
Q_FILE = Path("example1/ixpe01250401_det1_evt2_v03_Q-11bins.pha")
U_FILE = Path("example1/ixpe01250401_det1_evt2_v03_U-11bins.pha")
KEYS = [
    # Identity
    "TELESCOP", "INSTRUME", "DETNAM",
    # Response files
    "RESPFILE", "ANCRFILE", "BACKFILE", 
    # Stokes identifier
    "XFLT0001"
]

def header_summary(path, ext=1):
    print(f"\n--- Header summary: {path} (HDU {ext}) ---")
    with fits.open(path) as hdul:
        hdr = hdul[ext].header
        for k in KEYS:
            if k in hdr:
                print(f"{k:10s} = {hdr[k]}")
#    # Also show any keyword containing 'XFLT'
#    print("\nSelected matches (pattern search):")
#    for pat in ["XFLT"]:
#        hits = [k for k in hdr.keys() if pat in k]
#        if hits:
#            print(f"{pat}: {hits[:20]}{' ...' if len(hits)>20 else ''}")

header_summary(I_FILE, ext=1)
header_summary(Q_FILE, ext=1)
header_summary(U_FILE, ext=1)



--- Header summary: example1/ixpe01250401_det1_evt2_v03_I-11bins.pha (HDU 1) ---
TELESCOP   = IXPE
INSTRUME   = GPD
DETNAM     = DU1
RESPFILE   = ixpe_d1_20170101_alpha075_02.rmf
ANCRFILE   = ixpe01250401_det1_evt2_v03.arf
BACKFILE   = none
XFLT0001   = Stokes:0

--- Header summary: example1/ixpe01250401_det1_evt2_v03_Q-11bins.pha (HDU 1) ---
TELESCOP   = IXPE
INSTRUME   = GPD
DETNAM     = DU1
RESPFILE   = ixpe_d1_20170101_alpha075_02.rmf
ANCRFILE   = ixpe01250401_det1_evt2_v03.mrf
BACKFILE   = none
XFLT0001   = Stokes:1

--- Header summary: example1/ixpe01250401_det1_evt2_v03_U-11bins.pha (HDU 1) ---
TELESCOP   = IXPE
INSTRUME   = GPD
DETNAM     = DU1
RESPFILE   = ixpe_d1_20170101_alpha075_02.rmf
ANCRFILE   = ixpe01250401_det1_evt2_v03.mrf
BACKFILE   = none
XFLT0001   = Stokes:2


In [2]:
from pathlib import Path
from IPython.display import Markdown, display

XCM_PATH = Path("example1/example1-data.xcm")     # your .xcm file
xcm_text = XCM_PATH.read_text(errors="ignore")
display(Markdown(f"### XSPEC script: `{XCM_PATH}`"))
print(xcm_text)


### XSPEC script: `example1/example1-data.xcm`

model none
data none
setplot del all
query yes

data 1  ixpe01250401_det1_evt2_v03_I-11bins.pha
data 2  ixpe01250401_det2_evt2_v03_I-11bins.pha
data 3  ixpe01250401_det3_evt2_v03_I-11bins.pha
data 4  ixpe01250401_det1_evt2_v03_Q-11bins.pha
data 5  ixpe01250401_det2_evt2_v03_Q-11bins.pha
data 6  ixpe01250401_det3_evt2_v03_Q-11bins.pha
data 7  ixpe01250401_det1_evt2_v03_U-11bins.pha
data 8  ixpe01250401_det2_evt2_v03_U-11bins.pha
data 9  ixpe01250401_det3_evt2_v03_U-11bins.pha

setplot energy
ignore *:**-2.,8.01-**

setplot group 1-3
setplot group 4-6
setplot group 7-9

setplot co time off
setplot command csize 2
setplot command lw 5
setplot command lw 5 on 1..99

setplot co label top "plot ldata for I, Q, U"
setplot command r y 4e-4 4
cpd lda-iqu.eps/cps
pl lda
cpd /null

setplot co label top "plot polfrac"
setplot command r y 0. 0.15
setplot command log off
cpd polfrac.eps/cps
pl polfrac
cpd /null

setplot co label top "plot polangle"
setplot command r y 0. 30.
cpd polang.eps/cps
pl po

In [3]:
%%bash
cd example1
xspec - example1-data.xcm > example1-data.log 2>&1

In [4]:
from IPython.display import HTML, Image, display, Markdown

!convert -rotate 90 example1/lda-iqu.eps example1/lda-iqu.png
!convert -rotate 90 example1/polfrac.eps example1/polfrac.png
!convert -rotate 90 example1/polang.eps example1/polang.png

display(Markdown("## XSPEC polarization data plots: example #1 - 4U 1630-472"))
#display(Image(filename="lda-iqu.png"))
display(HTML(
'<div style="display:flex;">'
'<img src="example1/lda-iqu.png" width="300">'
'<img src="example1/polfrac.png" width="300">'
'<img src="example1/polang.png" width="300">'
'</div>'
))

## XSPEC polarization data plots: example #1 - 4U 1630-472

In [5]:
%%bash
cd example2
xspec - example2-data.xcm > example2-data.log 2>&1

In [6]:
from IPython.display import HTML, Image, display, Markdown

!convert -rotate 90 example2/lda-i.eps example2/lda-i.png
!convert -rotate 90 example2/da-qu.eps example2/da-qu.png
!convert -rotate 90 example2/polfrac.eps example2/polfrac.png
!convert -rotate 90 example2/polang.eps example2/polang.png

display(Markdown("## XSPEC polarization data plots: example #2 - Cyg X-3"))
#display(Image(filename="lda-iqu.png"))
display(HTML(
'<div style="display:flex;">'
'<img src="example2/lda-i.png" width="300">'
'<img src="example2/da-qu.png" width="300">'
'<img src="example2/polfrac.png" width="300">'
'<img src="example2/polang.png" width="300">'
'</div>'
))

## XSPEC polarization data plots: example #2 - Cyg X-3

## Polarization model types in XSPEC

One has to have polarization data sets for all Stokes spectra (I, Q, U) loaded into XSPEC to see the model prediction for polarization (Q, U, polfrac and polangle). 

### 1) Simple analytical polarization models
These operate on a flux-only spectral model (they describe polarization behavior without changing the spectral shape):
- `polconst` → constant PD and PA
- `pollin` → linear energy dependence of PD and PA
- `polpow` → power-law energy dependence of PD and PA

Useful for:
- quick empirical descriptions
- testing PD/PA trends

---

#### **[pollin: Linearly dependent polarization](https://heasarc.gsfc.nasa.gov/docs/software/xspec/manual/node219.html)**

This multiplicative model applies a polarization with a linear dependence on energy. The factor applied depends on which Stokes parameter the spectrum is for. The Stokes parameter is determined using an `XFLTnnnn` keyword with values "Stokes:0", "Stokes:1", or "Stokes:2" for I, Q and U, respectively

The fraction and angle are determined by

$$P(E) = \mathrm{A1} + (E - 1.0) \mathrm{\times} \mathrm{Aslope}$$
$$\psi(E) = \mathrm{psi1} + (E - 1.0) \mathrm{\times} \mathrm{psislope}$$

The parameters are :

- par1 = A1, the polarization fraction at 1 keV
- par2 = Aslope, the polarization fraction slope  
- par3 = psi1, the polarization angle at 1 keV (degrees)
- par4 = psislope, the polarization angle slope  

For a given underlying spectral intensity, I(E), the Stokes parameters are:

$$Q(E) = I(E) \, P(E) \cos\big(2\psi(E)\big)$$
$$U(E) = I(E) \, P(E) \sin\big(2\psi(E)\big)$$

The intensity, I(E), itself is not modified by `pollin`.

---


### 2) Table models for Stokes spectra
These contain precomputed Stokes spectra I, Q, U in OGIP FITS table form, where for each parameter set three rows with the three Stokes spectra are given. Further, the `XFXP0001`, `XFXP0002` and `XFXP0003` keywords need to be defined with values `Stokes:0`, `Stokes:1` and `Stokes:2`, respectively.

Advantages:
- physically motivated radiative transfer
- thus they constrain physical model and its parameters, e.g. geometry of the system

Python script for creating XSPEC polarization table models from ASCII tables:  
[https://github.com/mbursa/xspec-table-models](https://github.com/mbursa/xspec-table-models)
- It converts ASCII Stokes tables into OGIP-compliant XSPEC table models.
- It enables using custom radiative transfer outputs as XSPEC polarization table models.
- Useful if one computes I/Q/U grids externally (e.g. with KerrC, MONK or other codes).   

### 3) Advanced analytical physical models


* **Distant reflection tables**
  - Reflection from a distant disc: [https://github.com/jpodgorny/xsstokes_disc](https://github.com/jpodgorny/xsstokes_disc).
  - Reflection from a distant torus: [https://github.com/jpodgorny/xsstokes_torus](https://github.com/jpodgorny/xsstokes_torus).
  - Additional work on extended distant-reflector shapes is in construction.
  - References:
    - [Podgorný et al. (2024) MNRAS, 530, 2608](https://ui.adsabs.harvard.edu/abs/2024MNRAS.530.2608P)
    - [Podgorný (2025) A&A, 702, A43](https://ui.adsabs.harvard.edu/abs/2025A\%26A...702A..43P)
* **Relativistic models**
  - KYNSTOKES: [https://projects.asu.cas.cz/dovciak/kynstokes](https://projects.asu.cas.cz/dovciak/kynstokes)
    - Relativistic reflection from black-hole accretion disc including polarization transport.
    - [Podgorný et al. (2023) MNRAS, 524, 3853](https://ui.adsabs.harvard.edu/abs/2023MNRAS.524.3853P)
  - KYNBB / KYNBBRR
    - KYNBB is included in KYN model package: [https://projects.asu.cas.cz/stronggravity/kyn#kynbb](https://projects.asu.cas.cz/stronggravity/kyn#kynbb)
      - Thermal emission from accretion disc including relativistic polarization transport.
      - Standard Novikov-Thorne accretion disc.
      - Chandrasekhar approximation for local polarization properties.
      - [Dovčiak et al. (2008) MNRAS, 391, 32](https://ui.adsabs.harvard.edu/abs/2008MNRAS.391...32D)
    - KYNBBRR: thermal disc emission including accretion disc self-irradiation.
      - testing phase, will become publicly available this year
      - [Taverna et al. (2020) MNRAS, 391, 32](https://ui.adsabs.harvard.edu/abs/2020MNRAS.493.4960T)
      - [Mikušincová et al. (2023) MNRAS, 519, 6138](https://ui.adsabs.harvard.edu/abs/2023MNRAS.519.6138M)


## Using pollin model (practical overview)

In [7]:
from pathlib import Path
from IPython.display import Markdown, display

XCM_PATH = Path("example1/example1-data-model.xcm")     # your .xcm file
xcm_text = XCM_PATH.read_text(errors="ignore")
display(Markdown(f"### XSPEC script: `{XCM_PATH}`"))
print(xcm_text)


### XSPEC script: `example1/example1-data-model.xcm`

model none
data none
setplot del all
query yes

data 1:1  ixpe01250401_det1_evt2_v03_I-30bins.pha
data 1:2  ixpe01250401_det1_evt2_v03_Q-30bins.pha
data 1:3  ixpe01250401_det1_evt2_v03_U-30bins.pha

data 2:4  ixpe01250401_det2_evt2_v03_I-30bins.pha
data 2:5  ixpe01250401_det2_evt2_v03_Q-30bins.pha
data 2:6  ixpe01250401_det2_evt2_v03_U-30bins.pha

data 3:7  ixpe01250401_det3_evt2_v03_I-30bins.pha
data 3:8  ixpe01250401_det3_evt2_v03_Q-30bins.pha
data 3:9  ixpe01250401_det3_evt2_v03_U-30bins.pha

setplot energy
ignore *:**-2.,8.01-**

model  constant(pollin*TBabs*diskbb)
              1      -0.01          0          0      1e+10      1e+10
      0.0312801       0.01          0          0          1          1
       0.014437       0.01         -5         -5          5          5
         17.852       0.01        -90        -90         90         90
              0      -0.01         -5         -5          5          5
        8.31344      0.001          0          0     100000      1e+

In [8]:
%%bash
cd example1
xspec - example1-data-model.xcm > example1-data-model.log 2>&1

In [9]:
from IPython.display import HTML, Image, display, Markdown

!convert -rotate 90 example1/eufs-model.eps example1/eufs-model.png
!convert -rotate 90 example1/polfrac-model.eps example1/polfrac-model.png
!convert -rotate 90 example1/polang-model.eps example1/polang-model.png

display(Markdown("## XSPEC polarization data plots with model: example #1 - 4U 1630-472"))
display(HTML(
'<div style="display:flex;">'
'<img src="example1/eufs-model.png" width="300">'
'<img src="example1/polfrac-model.png" width="300">'
'<img src="example1/polang-model.png" width="300">'
'</div>'
))

## XSPEC polarization data plots with model: example #1 - 4U 1630-472

In [10]:
%%bash
cd example1
xspec - example1-data-model-11bins.xcm > example1-data-model-11bins.log 2>&1

In [11]:
from IPython.display import HTML, Image, display, Markdown

!convert -rotate 90 example1/lda-iqu-model-11bins.eps example1/lda-iqu-model-11bins.png
!convert -rotate 90 example1/polfrac-model-11bins.eps example1/polfrac-model-11bins.png
!convert -rotate 90 example1/polang-model-11bins.eps example1/polang-model-11bins.png

display(Markdown("## XSPEC polarization data plots with model: example #1 - 4U 1630-472"))
display(HTML(
'<div style="display:flex;">'
'<img src="example1/lda-iqu-model-11bins.png" width="300">'
'<img src="example1/polfrac-model-11bins.png" width="300">'
'<img src="example1/polang-model-11bins.png" width="300">'
'</div>'
))

## XSPEC polarization data plots with model: example #1 - 4U 1630-472

## Slab reflection Stokes tables (practical overview)

Power-law illumination tables, `STOKES`:
- [Podgorný et al. (2022), MNRAS 510, 4723](https://ui.adsabs.harvard.edu/abs/2022MNRAS.510.4723P)

Black-body illumination tables, `STOKESBB`:
- [Podgorný et al. (2026), A&A submitted](https://ui.adsabs.harvard.edu/abs/2025arXiv250723687P)

Repository: [https://github.com/jpodgorny/stokes_tables](https://github.com/jpodgorny/stokes_tables)

Model parameters include:
- photon index Γ or black-body temperature T
- ionization ξ
- incident, emission and azimuthal angles
- polarization of illumination

#### XSPEC usage of slab Stokes tables

- `model polrot * atable{stokes_unpol_reduced-v2.fits}`

Notes:
- `polrot` is used to rotate Stokes parameters to match the system orientation.
- Table filenames for power-law illumination: `stokes_unpol_iso-v2.fits`, `stokes_unpol-v2.fits`, `stokes_vrpol-v2.fits`, `stokes_45deg-v2.fits`.
- Table filenames for black-body illumination: `stokesBB_unpol-v2.fits`, `stokesBB_vrpol-v2.fits`, `stokesBB_45deg-v2.fits`.
- Reduced table versions are available suitable for IXPE.


#### How Stokes rotation works

If the polarization reference frame is rotated by an angle, $\psi$, the linear Stokes parameters transform as:

$$Q' = Q \cos(2\psi) + U \sin(2\psi)$$
$$U' = -Q \sin(2\psi) + U \cos(2\psi)$$

The intensity, I, remains unchanged. The factor of 2 appears because polarization is defined modulo 180°.


## Constructing arbitrary illumination polarization from table models

The slab tables provide spectra for:

- Unpolarised illumination: S(0,-)
- 100% polarised illumination at 0°: S(100\%,0°)
- 100% polarised illumination at 45°: S(100\%,45°)

These provide the basis vectors in Stokes space. Arbitrary polarization states are then built as linear combinations. A general illumination with polarization degree, P, and angle, $\chi$, can be constructed as:

$$S(P,\chi)=S(0,-)+P\Big(\big[S(100\%,0^\circ)-S(0,-)\big]\cos2\chi+\big[S(100\%,45^\circ)-S(0,-)\big]\sin2\chi\Big)$$

Special case (polarization of illumination perpendicular or parallel with the slab):

$$S(P)=S(0,-)+P\big[S(100\%,0^\circ)-S(0,-)\big]$$

One can then use `mdefine` in XSPEC to define models for arbitrary polarization:
* `mdefine stiso atable{stokes_unpol_iso-v2.fits}(PhoIndex, Xi, cosd(Thetae), z) : add`
* similarly for other polarization models `stunp`, `stvrp` and `st45d`
* combined model for alligned polarization: `stpol = stunp + P * [ stvrp - stunp ]`
* combined model for arbitrary polarization: `stokes = stunp + P * { [ stvrp - stunp ] * cos 2χ + [ st45d - stunp ] * sin 2χ }`
* Usage examples:
   - `model polrot * stiso`
   - `model polrot * stpol`
   - `model polrot * stokes`
* **Note that XSPEC does not allow for definition of parameter limits with mdefine, thus one has to set them manually before using the defined models!**


In [12]:
from pathlib import Path
from IPython.display import Markdown, display

XCM_PATH = Path("STOKES_model_definitions.xcm")     # your .xcm file
xcm_text = XCM_PATH.read_text(errors="ignore")
display(Markdown(f"### XSPEC script: `{XCM_PATH}`"))
print(xcm_text)


### XSPEC script: `STOKES_model_definitions.xcm`

# let's set the directory with the tables
# note that the directory definition should NOT end with a slash!
#set STOKESDIR /home/dovciak/data
set STOKESDIR .
# let's define the tables to be used (e.g. the full ones or the reduced ones)
set UNPOL stokes_unpol-v2.fits
set VRPOL stokes_vrpol-v2.fits
set 45POL stokes_45deg-v2.fits
set UNPISO stokes_unpol_iso-v2.fits
# define the STOKES polarisation models, see Podgorny et al. (2022), Podgorny (2023)
mdefine stunp atable{$STOKESDIR/$UNPOL}(PhoIndex, Xi, cosd(Thetai), Phi, cosd(Thetae), z) : add
mdefine stvrp atable{$STOKESDIR/$VRPOL}(PhoIndex, Xi, cosd(Thetai), Phi, cosd(Thetae), z) : add
mdefine st45d atable{$STOKESDIR/$45POL}(PhoIndex, Xi, cosd(Thetai), Phi, cosd(Thetae), z) : add
mdefine stiso atable{$STOKESDIR/$UNPISO}(PhoIndex, Xi, cosd(Thetae), z) : add
mdefine stpol stunp(PhoIndex, Xi, Thetai, Phi, Thetae, z)+PolFrac*((stvrp(PhoIndex, Xi, Thetai, Phi, Thetae, z)-stunp(PhoIndex, Xi, Thetai, Phi, Thetae, z))) : add
mdefine stokes stunp

In [13]:
from pathlib import Path
from IPython.display import Markdown, display

XCM_PATH = Path("example2/example2-data-model.xcm")     # your .xcm file
xcm_text = XCM_PATH.read_text(errors="ignore")
display(Markdown(f"### XSPEC script: `{XCM_PATH}`"))
print(xcm_text)


### XSPEC script: `example2/example2-data-model.xcm`

model none
data none
setplot del all
query yes

data 1:1 ixpe02001899_det1_evt2_v01_I-30bins.pha
data 1:2 ixpe02001899_det1_evt2_v01_Q-30bins.pha
data 1:3 ixpe02001899_det1_evt2_v01_U-30bins.pha

data 2:4 ixpe02001899_det2_evt2_v01_I-30bins.pha
data 2:5 ixpe02001899_det2_evt2_v01_Q-30bins.pha
data 2:6 ixpe02001899_det2_evt2_v01_U-30bins.pha

data 3:7 ixpe02001899_det3_evt2_v01_I-30bins.pha
data 3:8 ixpe02001899_det3_evt2_v01_Q-30bins.pha
data 3:9 ixpe02001899_det3_evt2_v01_U-30bins.pha

setplot energy
ignore *:**-2.,8.01-**

model  polrot*constant*TBabs(atable{stokes_unpol-v2.fits} + atable{stokes_unpol-v2.fits})
        12.4917        0.1       -180       -180        180        180
              1       -0.1        0.9        0.9        1.1        1.1
        7.83595        0.1          0          0     100000      1e+06
            2.7       -0.1        1.4        1.4          3          3
         6759.4         10          5          5      20000      20000
      0.0871557      -0.

In [14]:
%%bash
cd example2
xspec - example2-data-model.xcm > example2-data-model.log 2>&1

In [15]:
from IPython.display import HTML, Image, display, Markdown

!convert -rotate 90 example2/eufs-model.eps example2/eufs-model.png
!convert -rotate 90 example2/polfrac-model.eps example2/polfrac-model.png
!convert -rotate 90 example2/polang-model.eps example2/polang-model.png

display(Markdown("## XSPEC polarization data plots with model: example #2 - Cyg X-3"))
display(HTML(
'<div style="display:flex;">'
'<img src="example2/eufs-model.png" width="300">'
'<img src="example2/polfrac-model.png" width="300">'
'<img src="example2/polang-model.png" width="300">'
'</div>'
))

## XSPEC polarization data plots with model: example #2 - Cyg X-3